# Terminal Service Process Spawn
Detects a process spawned by the terminal service server process (this could be an indicator for an exploitation of CVE-2019-0708)

## Rule Content
```
- title: Terminal Service Process Spawn
  id: 1012f107-b8f1-4271-af30-5aed2de89b39
  status: experimental
  description: Detects a process spawned by the terminal service server process (this
    could be an indicator for an exploitation of CVE-2019-0708)
  references:
  - https://securingtomorrow.mcafee.com/other-blogs/mcafee-labs/rdp-stands-for-really-do-patch-understanding-the-wormable-rdp-vulnerability-cve-2019-0708/
  author: Florian Roth
  date: 2019/05/22
  tags:
  - car.2013-07-002
  logsource:
    product: windows
    category: process_creation
    service: null
  detection:
    selection:
      ParentCommandLine: '*\svchost.exe*termsvcs'
    filter:
      Image: '*\rdpclip.exe'
    condition: selection and not filter
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_parent_command_line.keyword:*\\svchost.exe*termsvcs AND (NOT (process_path.keyword:*\\rdpclip.exe)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()